In [1]:
print('Importing packages...')
import os
import pandas
import numpy as np
import time
pandas.set_option('display.max_colwidth', -1)
debug = 1

# Directories to look in
thisdir = '/home/idies/workspace/Storage/raddick/Baltimore/community_reinvestment_act/'
data_dir = '/home/idies/workspace/Temporary/raddick/cra_scratch_final/'
jobs_dir = data_dir + 'lodes_wac/'
census_dir = data_dir + 'acs5/'
#baltimore_dir = thisdir + 'baltimore/'
code_lookup_dir = thisdir + 'code_guide_lookups/'
inflation_dir = '/home/idies/workspace/Storage/raddick/Baltimore/community_reinvestment_act/datasets/inflation/'
extrasdir = '/home/idies/workspace/Storage/raddick/census/extras/'

city_data_dir = thisdir + 'city_data/'

g = 0  # keep track of grand total of processing time
os.chdir(thisdir)
print('Now in directory: {0:}'.format(os.getcwd()))
print('ok')

Importing packages...
Now in directory: /home/idies/workspace/Storage/raddick/Baltimore/community_reinvestment_act
ok


In [2]:
state_codes_df = pandas.read_csv(extrasdir+'statecodes.csv')
state_codes_df = state_codes_df.set_index('STUSAB')
states = state_codes_df.index.values.tolist()
states = sorted([x.lower() for x in states if x not in ('MD', 'AS', 'GU', 'MP', 'PR', 'UM', 'VI')])

city = 'St. Louis'

if (city == 'Baltimore'):
    thestate = 24
    state_abbrev = 'md'
    thecounty = 510
    shapefile_dir = '/home/idies/workspace/Storage/raddick/Baltimore/shapefiles/'
    cityname_file = 'baltimore'
    print('Selected {0:}!'.format(city))
if (city == 'St. Louis'):
    thestate = 29
    state_abbrev = 'mo'
    thecounty = 510
    shapefile_dir = '/home/idies/workspace/Storage/raddick/Baltimore/shapefiles/'
    cityname_file = 'st_louis'
    print('Selected {0:}!'.format(city))
else:
    print('ERROR: Select city!')
#state_codes_df



Selected St. Louis!


# Get flatfile loans data for selected city, process, save to file

In [3]:
s = time.time()
print('reading nationwide loans data...')
loans_df = pandas.read_csv(data_dir+'loans.csv', encoding='utf-8', low_memory=False, index_col='rownumber')

#print('   assessment areas...')
#assessment_areas_df = pandas.read_csv(data_dir+'assessment_areas.csv', encoding='utf-8', low_memory=False, index_col='rownumber')
#print('   census tracts...')
#tracts_df = pandas.read_csv(data_dir+'tracts.csv', encoding='utf-8', low_memory=False, index_col='rownumber')
#print('   assessment areas...')
#city_assessment_areas_df = assessment_areas_df[(assessment_areas_df['state'] == thestate) & (assessment_areas_df['county'] == thecounty)]
#print('   census tracts...')

print('keeping only business loan originations in {0:}...'.format(city))
city_loans_df = loans_df[(loans_df['state'] == thestate) & (loans_df['county'] == thecounty)]
city_loans_df = city_loans_df[city_loans_df['loan_type'] == 4] # Small business loans only
city_loans_df = city_loans_df[city_loans_df['action_taken_type'] == 1] # Originations only

print('calculating total loans...')
city_loans_df = city_loans_df.assign(nLoans = city_loans_df['nLoans1'] + city_loans_df['nLoans100k'] + city_loans_df['nLoans250k'])
city_loans_df = city_loans_df.assign(amtLoans = city_loans_df['amtLoans1'] + city_loans_df['amtLoans100k'] + city_loans_df['amtLoans250k'])

print('keeping only respondents with loans...')
city_loans_df = city_loans_df[city_loans_df['nLoans'] > 0]

# IF NO ASSESSMENT AREA NUMBER, there are 33 problematic areas...
### BUT investigation shows they are all duplicates!
#city_loans_df[(city_loans_df['nLoans'] > 0) & (city_loans_df['assessment_area_number'].isnull())].groupby(['respondentID', 'activity_year', 'income_group_code']).size().sort_values(ascending=False)[0:33].sort_index()
print('dropping duplicate rows...')
city_loans_df = city_loans_df[
    (city_loans_df['assessment_area_number'].isnull())
].drop_duplicates(['respondentID', 'activity_year', 'income_group_total'])

print('looking up codes for institution names, income groups, CRA levels...')
respondents_df = pandas.read_csv(code_lookup_dir+'respondentid.csv', encoding='utf-8', low_memory=False)
city_loans_df = city_loans_df.reset_index().merge(respondents_df, how='left', on='respondentID').set_index('rownumber')

city_loans_df = city_loans_df.assign(agency = np.nan)
city_loans_df.loc[city_loans_df['agency_code'] == 1, 'agency'] = 'OCC'
city_loans_df.loc[city_loans_df['agency_code'] == 2, 'agency'] = 'FRS'
city_loans_df.loc[city_loans_df['agency_code'] == 3, 'agency'] = 'FDIC'
city_loans_df.loc[city_loans_df['agency_code'] == 4, 'agency'] = 'OTS'

city_loans_df = city_loans_df.rename(columns = {'income_group_total': 'income_group_code'})
city_loans_df = city_loans_df.assign(income_group = np.nan)

city_loans_df.loc[city_loans_df['income_group_code'] == 1, 'income_group'] = '< 10% of Median Family Income (MFI)'
city_loans_df.loc[city_loans_df['income_group_code'] == 2, 'income_group'] = '10% to 20% of MFI'
city_loans_df.loc[city_loans_df['income_group_code'] == 3, 'income_group'] = '20% to 30% of MFI'
city_loans_df.loc[city_loans_df['income_group_code'] == 4, 'income_group'] = '30% to 40% of MFI'
city_loans_df.loc[city_loans_df['income_group_code'] == 5, 'income_group'] = '40% to 50% of MFI'
city_loans_df.loc[city_loans_df['income_group_code'] == 6, 'income_group'] = '50% to 60% of MFI'
city_loans_df.loc[city_loans_df['income_group_code'] == 7, 'income_group'] = '60% to 70% of MFI'
city_loans_df.loc[city_loans_df['income_group_code'] == 8, 'income_group'] = '70% to 80% of MFI'
city_loans_df.loc[city_loans_df['income_group_code'] == 9, 'income_group'] = '80% to 90% of MFI'
city_loans_df.loc[city_loans_df['income_group_code'] == 10, 'income_group'] = '90% to 100% of MFI'
city_loans_df.loc[city_loans_df['income_group_code'] == 11, 'income_group'] = '100% to 110% of MFI'
city_loans_df.loc[city_loans_df['income_group_code'] == 12, 'income_group'] = '110% to 120% of MFI'
city_loans_df.loc[city_loans_df['income_group_code'] == 13, 'income_group'] = '> 120% of MFI'

# Get levels (low, moderate, middle, upper)
city_loans_df = city_loans_df.assign(cra_level = np.nan)
city_loans_df.loc[(city_loans_df['income_group_code'] >= 1) & (city_loans_df['income_group_code'] <= 5), 'cra_level'] = 'low'
city_loans_df.loc[(city_loans_df['income_group_code'] >= 6) & (city_loans_df['income_group_code'] <= 8), 'cra_level'] = 'moderate'
city_loans_df.loc[(city_loans_df['income_group_code'] >= 9) & (city_loans_df['income_group_code'] <= 12), 'cra_level'] = 'middle'
city_loans_df.loc[(city_loans_df['income_group_code'] == 13), 'cra_level'] = 'upper'
city_loans_df.loc[(city_loans_df['income_group_code'] == 14), 'cra_level'] = 'unknown'

print('calculating working loans...')
city_loans_df = city_loans_df.assign(avgSmallLoan = city_loans_df['amtLoans1'] / city_loans_df['nLoans1'])
city_loans_df = city_loans_df.assign(nWorkingLoans = np.nan)
city_loans_df = city_loans_df.assign(amtWorkingLoans = np.nan)
city_loans_df.loc[city_loans_df['avgSmallLoan'] < 10000, 
                           'nWorkingLoans'] = city_loans_df['nLoans'][city_loans_df['avgSmallLoan'] < 10000] - city_loans_df['nLoans1'][city_loans_df['avgSmallLoan'] < 10000]
city_loans_df.loc[city_loans_df['avgSmallLoan'] >= 10000, 
                           'nWorkingLoans'] = city_loans_df['nLoans'][city_loans_df['avgSmallLoan'] >= 10000]
city_loans_df.loc[city_loans_df['avgSmallLoan'] < 10000, 
                           'amtWorkingLoans'] = city_loans_df['amtLoans'][city_loans_df['avgSmallLoan'] < 10000] - city_loans_df['amtLoans1'][city_loans_df['avgSmallLoan'] < 10000]
city_loans_df.loc[city_loans_df['avgSmallLoan'] >= 10000, 
                           'amtWorkingLoans'] = city_loans_df['amtLoans'][city_loans_df['avgSmallLoan'] >= 10000]

# If lender is American Expres, set working loans to zero
city_loans_df.loc[city_loans_df['institution_name'].apply(lambda x: 'american express' in str(x).lower()), 'nWorkingLoans'] = 0
city_loans_df.loc[city_loans_df['institution_name'].apply(lambda x: 'american express' in str(x).lower()), 'amtWorkingLoans'] = 0


print('setting index...')
city_loans_df = city_loans_df.set_index(['respondentID', 'activity_year', 'assessment_area_number', 'income_group_code'])

money_columns = ['amtLoans1', 'amtLoans100k', 'amtLoans250k', 'amtLoansToSmallest']
money_columns += ['amtLoans', 'amtWorkingLoans']

print('correcting for inflation...')
for x in money_columns:
    city_loans_df.loc[:, x] = city_loans_df[x].fillna(0)

cpi_1913_2017_df = pandas.read_csv(inflation_dir+'cpi-1913-2017.csv', index_col='Year')
cpi_annual_s = cpi_1913_2017_df['Jan']
cpi_annual_s.name = 'rawfactor'
value_in_2017 = cpi_annual_s.loc[2017]

annual_inflator_s = 1 / (cpi_annual_s / value_in_2017)
annual_inflator_s.index.name = 'activity_year'

print('inflating pre-2017 monetary values...')
inflate_these_df = city_loans_df[money_columns]
newcolnames = [x+'_adj' for x in inflate_these_df.columns.tolist()]
inflate_these_df.columns = newcolnames
inflate_these_df = inflate_these_df.reset_index().set_index('activity_year').join(annual_inflator_s).reset_index().set_index(['respondentID', 'activity_year', 'assessment_area_number', 'income_group_code'])
for x in newcolnames:
    inflate_these_df.loc[:, x] = inflate_these_df[x] * inflate_these_df['rawfactor']
inflate_these_df = inflate_these_df.drop('rawfactor', axis=1)
city_loans_df = city_loans_df.join(inflate_these_df)

writefilename = city_data_dir+'{0:}_loans.csv'.format(cityname_file)
print('Saving data to {0:}...'.format(writefilename))
city_loans_df.to_csv(writefilename, encoding='utf-8')

e = time.time()
g = g + (e-s)
print('Saved {0:,.0f} rows of loan data for {1:} in {2:,.0f} seconds!'.format(len(city_loans_df), city, e-s))


reading nationwide loans data...


/home/idies/miniconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


keeping only business loan originations in St. Louis...
calculating total loans...
keeping only respondents with loans...
dropping duplicate rows...
looking up codes for institution names, income groups, CRA levels...
calculating working loans...
setting index...
correcting for inflation...
inflating pre-2017 monetary values...


/home/idies/miniconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2817: PerformanceWarning: indexing past lexsort depth may impact performance.
  raw_cell, store_history, silent, shell_futures)


Saving data to /home/idies/workspace/Storage/raddick/Baltimore/community_reinvestment_act/city_data/st_louis_loans.csv...
Saved 1,652 rows of loan data for St. Louis in 66 seconds!


# Get data aggregated by lenders

In [4]:
s = time.time()
print('reading nationwide data...')
agg_lenders_df = pandas.read_csv(data_dir+'agg_lenders.csv', encoding='utf-8', low_memory=False, index_col='rownumber')
e = time.time()
g = g + (e-s)
print('read {0:,.0f} nationwide lender-years in {1:,.0f} seconds...'.format(len(agg_lenders_df), e-s))

s = time.time()
print('selecting originated business loans in {0:}...'.format(city))
agg_lenders_df = agg_lenders_df[agg_lenders_df['loan_type'] == 4]
agg_lenders_df = agg_lenders_df[agg_lenders_df['action_taken_type'] == 1]
city_agg_lenders_df = agg_lenders_df[
    (agg_lenders_df['state'] == thestate)
    & (agg_lenders_df['county'] == thecounty)
]

print('looking up institution names from respondentID...')
respondents_df = pandas.read_csv(code_lookup_dir+'respondentid.csv', encoding='utf-8', low_memory=False)
city_agg_lenders_df = city_agg_lenders_df.reset_index().merge(respondents_df, how='left', on='respondentID').set_index('rownumber')

print('looking up agency names from agency_code...')
city_agg_lenders_df = city_agg_lenders_df.assign(agency = np.nan)
city_agg_lenders_df.loc[city_agg_lenders_df['agency_code'] == 1, 'agency'] = 'OCC'
city_agg_lenders_df.loc[city_agg_lenders_df['agency_code'] == 2, 'agency'] = 'FRS'
city_agg_lenders_df.loc[city_agg_lenders_df['agency_code'] == 3, 'agency'] = 'FDIC'
city_agg_lenders_df.loc[city_agg_lenders_df['agency_code'] == 4, 'agency'] = 'OTS'

print('setting index to respondent-year...')
city_agg_lenders_df = city_agg_lenders_df.set_index(['respondentID', 'activity_year'])
city_agg_lenders_df = city_agg_lenders_df.sort_index()

print('correcting for inflation...')
money_columns = ['amtLoans', 'amtLoansToSmallest']

for x in money_columns:
    city_agg_lenders_df.loc[:, x] = city_agg_lenders_df[x].fillna(0)

cpi_1913_2017_df = pandas.read_csv(inflation_dir+'cpi-1913-2017.csv', index_col='Year')
cpi_annual_s = cpi_1913_2017_df['Jan']
cpi_annual_s.name = 'rawfactor'
value_in_2017 = cpi_annual_s.loc[2017]

annual_inflator_s = 1 / (cpi_annual_s / value_in_2017)
print('inflating pre-2017 monetary values...')
inflate_these_df = city_agg_lenders_df[money_columns]
newcolnames = [x+'_adj' for x in inflate_these_df.columns.tolist()]
inflate_these_df.columns = newcolnames

city_agg_lenders_df = city_agg_lenders_df.reset_index().merge(inflate_these_df.reset_index(), how='left', on=['respondentID','activity_year']).set_index(['respondentID','activity_year'])

writefilename = city_data_dir+'{0:}_agg_lenders.csv'.format(cityname_file)
print('Saving data to {0:}...'.format(writefilename))
city_agg_lenders_df.to_csv(writefilename, encoding='utf-8')

e = time.time()
g = g + (e-s)
print('Saved {0:,.0f} lender-years in {1:} in {2:,.0f} seconds!'.format(len(city_agg_lenders_df), city, e-s))


reading nationwide data...
read 2,166,112 nationwide lender-years in 6 seconds...
selecting originated business loans in St. Louis...
looking up institution names from respondentID...
looking up agency names from agency_code...
setting index to respondent-year...
correcting for inflation...
inflating pre-2017 monetary values...
Saving data to /home/idies/workspace/Storage/raddick/Baltimore/community_reinvestment_act/city_data/st_louis_agg_lenders.csv...
Saved 798 lender-years in St. Louis in 1 seconds!


# Get data aggregated by census tracts

This is the basis for most of our analysis

# Get data for loans aggregated by census tract & year

In [5]:
s = time.time()
print('reading nationwide data...')
agg_loans_df = pandas.read_csv(data_dir+'agg_loans.csv', encoding='utf-8', low_memory=False, index_col='rownumber')
e = time.time()
g = g + (e-s)
print('read {0:,.0f} nationwide tract-years in {1:,.0f} seconds...'.format(len(agg_loans_df), e-s))

s = time.time()
print('selecting originated business loans in {0:}...'.format(city))
# Keep only business loans
agg_loans_df = agg_loans_df[agg_loans_df['loan_type'] == 4]
# Keep only loan originations
agg_loans_df = agg_loans_df[agg_loans_df['action_taken_type'] == 1]

# Keep only loans in the city of interest
city_agg_loans_df = agg_loans_df[
    (agg_loans_df['state'] == thestate)
    & (agg_loans_df['county'] == thecounty)
]

print('looking up income group names from income_group_total...')
city_agg_loans_df = city_agg_loans_df.rename(columns = {'income_group_total': 'income_group_code'})
city_agg_loans_df = city_agg_loans_df.assign(income_group = np.nan)

city_agg_loans_df.loc[city_agg_loans_df['income_group_code'] == 1, 'income_group'] = '< 10% of Median Family Income (MFI)'
city_agg_loans_df.loc[city_agg_loans_df['income_group_code'] == 2, 'income_group'] = '10% to 20% of MFI'
city_agg_loans_df.loc[city_agg_loans_df['income_group_code'] == 3, 'income_group'] = '20% to 30% of MFI'
city_agg_loans_df.loc[city_agg_loans_df['income_group_code'] == 4, 'income_group'] = '30% to 40% of MFI'
city_agg_loans_df.loc[city_agg_loans_df['income_group_code'] == 5, 'income_group'] = '40% to 50% of MFI'
city_agg_loans_df.loc[city_agg_loans_df['income_group_code'] == 6, 'income_group'] = '50% to 60% of MFI'
city_agg_loans_df.loc[city_agg_loans_df['income_group_code'] == 7, 'income_group'] = '60% to 70% of MFI'
city_agg_loans_df.loc[city_agg_loans_df['income_group_code'] == 8, 'income_group'] = '70% to 80% of MFI'
city_agg_loans_df.loc[city_agg_loans_df['income_group_code'] == 9, 'income_group'] = '80% to 90% of MFI'
city_agg_loans_df.loc[city_agg_loans_df['income_group_code'] == 10, 'income_group'] = '90% to 100% of MFI'
city_agg_loans_df.loc[city_agg_loans_df['income_group_code'] == 11, 'income_group'] = '100% to 110% of MFI'
city_agg_loans_df.loc[city_agg_loans_df['income_group_code'] == 12, 'income_group'] = '110% to 120% of MFI'
city_agg_loans_df.loc[city_agg_loans_df['income_group_code'] == 13, 'income_group'] = '> 120% of MFI'

print('Adding CRA income levels (low/moderate/middle/upper/unknown)...')
# Get levels (low, moderate, middle, upper)
city_agg_loans_df = city_agg_loans_df.assign(cra_level = np.nan)
city_agg_loans_df.loc[(city_agg_loans_df['income_group_code'] >= 1) & (city_agg_loans_df['income_group_code'] <= 5), 'cra_level'] = 'low'
city_agg_loans_df.loc[(city_agg_loans_df['income_group_code'] >= 6) & (city_agg_loans_df['income_group_code'] <= 8), 'cra_level'] = 'moderate'
city_agg_loans_df.loc[(city_agg_loans_df['income_group_code'] >= 9) & (city_agg_loans_df['income_group_code'] <= 12), 'cra_level'] = 'middle'
city_agg_loans_df.loc[(city_agg_loans_df['income_group_code'] == 13), 'cra_level'] = 'upper'
city_agg_loans_df.loc[(city_agg_loans_df['income_group_code'] == 14), 'cra_level'] = 'unknown'

print('calculating total loans...')
city_agg_loans_df = city_agg_loans_df.assign(nLoans = city_agg_loans_df['nLoans1'] + city_agg_loans_df['nLoans100k'] + city_agg_loans_df['nLoans250k'])
city_agg_loans_df = city_agg_loans_df.assign(amtLoans = city_agg_loans_df['amtLoans1'] + city_agg_loans_df['amtLoans100k'] + city_agg_loans_df['amtLoans250k'])

print('calculating working loans...')
city_agg_loans_df = city_agg_loans_df.assign(avgSmallLoan = city_agg_loans_df['amtLoans1'] / city_agg_loans_df['nLoans1'])

city_agg_loans_df = city_agg_loans_df.assign(nWorkingLoans = np.nan)
city_agg_loans_df.loc[city_agg_loans_df['avgSmallLoan'] < 10000, 
                           'nWorkingLoans'] = city_agg_loans_df['nLoans'][city_agg_loans_df['avgSmallLoan'] < 10000] - city_agg_loans_df['nLoans1'][city_agg_loans_df['avgSmallLoan'] < 10000]
city_agg_loans_df.loc[city_agg_loans_df['avgSmallLoan'] >= 10000, 
                           'nWorkingLoans'] = city_agg_loans_df['nLoans'][city_agg_loans_df['avgSmallLoan'] >= 10000]

city_agg_loans_df = city_agg_loans_df.assign(amtWorkingLoans = np.nan)
city_agg_loans_df.loc[city_agg_loans_df['avgSmallLoan'] < 10000, 
                           'amtWorkingLoans'] = city_agg_loans_df['amtLoans'][city_agg_loans_df['avgSmallLoan'] < 10000] - city_agg_loans_df['amtLoans1'][city_agg_loans_df['avgSmallLoan'] < 10000]
city_agg_loans_df.loc[city_agg_loans_df['avgSmallLoan'] >= 10000, 
                           'amtWorkingLoans'] = city_agg_loans_df['amtLoans'][city_agg_loans_df['avgSmallLoan'] >= 10000]

city_agg_loans_df.sample(1).T#.groupby('population_classification').size()


# Each row is unique to census-tract and year, so set that as the index
city_agg_loans_df = city_agg_loans_df.set_index(['census_tract', 'activity_year'])
city_agg_loans_df = city_agg_loans_df.sort_index()

e = time.time()
g = g + (e-s)

print('Kept {0:,.0f} tract-years in {1:} in {2:,.2f} seconds!'.format(len(city_agg_loans_df), city, e-s))
city_agg_loans_df.head(1)

reading nationwide data...
read 2,645,400 nationwide tract-years in 11 seconds...
selecting originated business loans in St. Louis...
looking up income group names from income_group_total...
Adding CRA income levels (low/moderate/middle/upper/unknown)...
calculating total loans...
calculating working loans...
Kept 1,540 tract-years in St. Louis in 0.93 seconds!


,,loan_type,action_taken_type,state,county,msa,split_county_indicator,population_classification,income_group_code,nLoans1,amtLoans1,...,amtLoans250k,nLoansToSmallest,amtLoansToSmallest,income_group,cra_level,nLoans,amtLoans,avgSmallLoan,nWorkingLoans,amtWorkingLoans
census_tract,activity_year,,,,,,,,,,,,,,,,,,,,,
1011.0,2004,4,1,29,510,41180.0,N,S,103,25,128000,...,0,6,24000,NaN,NaN,26,303000,5120.0,1.0,175000.0


# Let's connect geographic aggregates to other geographic data

## First, get jobs data for this state and keep city jobs data

In [6]:
print('reading state jobs data for {0:}...'.format(state_abbrev.upper()))
s = time.time()
state_raw_jobs_df = pandas.read_csv(jobs_dir+'jobs_data_{0:}.csv'.format(state_abbrev), index_col='rownumber')
e = time.time()
g = g + (e-s)
print('Read {0:,.0f} rows in {1:,.0f} seconds...'.format(len(state_raw_jobs_df), e-s))
s = time.time()
city_raw_jobs_df = state_raw_jobs_df[
    (state_raw_jobs_df['w_geocode'].apply(lambda x: int(str(x)[0:2]) == thestate))
    & (state_raw_jobs_df['w_geocode'].apply(lambda x: int(str(x)[2:5]) == thecounty))
 ]

e = time.time()
g = g + (e-s)
print('Kept {0:,.0f} rows in {1:,.0f} seconds!'.format(len(city_raw_jobs_df), e-s))

reading state jobs data for MO...
Read 21,579,712 rows in 158 seconds...
Kept 1,458,186 rows in 103 seconds!


In [7]:
s = time.time()

print('getting census tracts from GEOIDs...')
# GeoID format is STATE+COUNTY+TRACT+BLOCK (2+3+6+4 = 15 characters)
city_raw_jobs_df = city_raw_jobs_df.assign(census_tract = pandas.to_numeric(city_raw_jobs_df['w_geocode'].apply(lambda x: str(x)[5:9] + '.' + str(x)[9:11]), errors='coerce'))

sum_columns = [x for x in city_raw_jobs_df.columns.tolist() if x not in ('w_geocode', 'createdate', 'year')]


city_jobs_df = pandas.DataFrame()

for i in range(2017, 2010, -1):
    city_jobs_i_df = city_raw_jobs_df[sum_columns][city_raw_jobs_df['year'] == i].groupby('census_tract', as_index=False).sum()
    if (i >= 2016):
        city_jobs_i_df = city_raw_jobs_df[sum_columns][city_raw_jobs_df['year'] == 2015].groupby('census_tract', as_index=False).sum()
    city_jobs_i_df = city_jobs_i_df.assign(year = i)
    city_jobs_df = pandas.concat((city_jobs_df, city_jobs_i_df), axis=0)

city_jobs_df = city_jobs_df.rename(columns={'year': 'activity_year'})
city_jobs_df = city_jobs_df.set_index(['census_tract', 'activity_year'])

print('joining jobs data onto loans data from 2011 to 2017...')
city_tracts_years_df = city_agg_loans_df.reset_index()[city_agg_loans_df.reset_index()['activity_year'] >= 2011].set_index(['census_tract', 'activity_year'])
city_tracts_years_df = city_tracts_years_df.join(city_jobs_df)

e = time.time()
print('Processed {0:,.0f} rows in {1:,.1f} seconds!'.format(len(city_tracts_years_df), e-s))
#city_tracts_years_df.sample(1).T

getting census tracts from GEOIDs...
joining jobs data onto loans data from 2011 to 2017...
Processed 749 rows in 10.2 seconds!


## What jobs columns do we want?

In [8]:
jobs_metadata_df = pandas.read_csv(code_lookup_dir+'wac_jobs_metadata.csv', encoding='utf-8', index_col='varnum')
jobs_metadata_df = jobs_metadata_df.set_index('variable')

jobs_columns = ['C000', 'CA01', 'CA02', 'CA03', 'CE01', 'CE02', 'CE03', 'CNS01']
jobs_columns += ['CNS02', 'CNS03', 'CNS04', 'CNS05', 'CNS06', 'CNS07', 'CNS08']
jobs_columns += ['CNS09', 'CNS10', 'CNS11', 'CNS12', 'CNS13', 'CNS14', 'CNS15']
jobs_columns += ['CNS16', 'CNS17', 'CNS18', 'CNS19', 'CNS20', 'CR01', 'CR02']
jobs_columns += ['CR03', 'CR04', 'CR05', 'CR07', 'CT01', 'CT02', 'CD01', 'CD02']
jobs_columns += ['CD03', 'CD04', 'CS01', 'CS02', 'CFA01', 'CFA02', 'CFA03']
jobs_columns += ['CFA04', 'CFA05', 'CFS01', 'CFS02', 'CFS03', 'CFS04', 'CFS05']

#for x in baltimore_agg_loans_df[jobs_columns].columns:
#    print('variable: {0:}\t\tdescription:{1:}'.format(x, jobs_metadata_df['description'][jobs_metadata_df.index == x].tolist()[0]))
    
jobs_columns_we_want = ['C000', 'CFS01']
for x in city_tracts_years_df[jobs_columns_we_want].columns:
    print('variable: {0:}\t\tdescription:{1:}'.format(x, jobs_metadata_df['description'][jobs_metadata_df.index == x].tolist()[0]))


variable: C000		description:Total number of jobs
variable: CFS01		description:Number of jobs for workers at firms with Firm Size: 0-19 Employees


## Get loans per job

In [9]:
#sbjobs_column = jobs_varnames_df[jobs_varnames_df['description'].apply(lambda x: '0-19' in x)].index.values[0]
#loans_columns = []
#city_tracts_years_df[sbjobs_column]
print('Calulating loans per job (total and with firm size 0-19)...')

city_tracts_years_df = city_tracts_years_df.assign(nLoans1_per_totaljob = city_tracts_years_df['nLoans1'] / city_tracts_years_df['C000'])
city_tracts_years_df = city_tracts_years_df.assign(amtLoans1_per_totaljob = city_tracts_years_df['amtLoans1'] / city_tracts_years_df['C000'])
city_tracts_years_df = city_tracts_years_df.assign(nLoans100k_per_totaljob = city_tracts_years_df['nLoans100k'] / city_tracts_years_df['C000'])
city_tracts_years_df = city_tracts_years_df.assign(amtLoans100k_per_totaljob = city_tracts_years_df['amtLoans100k'] / city_tracts_years_df['C000'])
city_tracts_years_df = city_tracts_years_df.assign(nLoans250k_per_totaljob = city_tracts_years_df['nLoans250k'] / city_tracts_years_df['C000'])
city_tracts_years_df = city_tracts_years_df.assign(amtLoans250k_per_totaljob = city_tracts_years_df['amtLoans250k'] / city_tracts_years_df['C000'])
city_tracts_years_df = city_tracts_years_df.assign(nLoansToSmallest_per_totaljob = city_tracts_years_df['nLoansToSmallest'] / city_tracts_years_df['C000'])
city_tracts_years_df = city_tracts_years_df.assign(amtLoansToSmallest_per_totaljob = city_tracts_years_df['amtLoansToSmallest'] / city_tracts_years_df['C000'])
city_tracts_years_df = city_tracts_years_df.assign(nLoans_per_totaljob = city_tracts_years_df['nLoans'] / city_tracts_years_df['C000'])
city_tracts_years_df = city_tracts_years_df.assign(amtLoans_per_totaljob = city_tracts_years_df['amtLoans'] / city_tracts_years_df['C000'])
city_tracts_years_df = city_tracts_years_df.assign(nWorkingLoans_per_totaljob = city_tracts_years_df['nWorkingLoans'] / city_tracts_years_df['C000'])
city_tracts_years_df = city_tracts_years_df.assign(amtWorkingLoans_per_totaljob = city_tracts_years_df['amtWorkingLoans'] / city_tracts_years_df['C000'])

city_tracts_years_df = city_tracts_years_df.assign(nLoans1_per_sbjob = city_tracts_years_df['nLoans1'] / city_tracts_years_df['CFS01'])
city_tracts_years_df = city_tracts_years_df.assign(amtLoans1_per_sbjob = city_tracts_years_df['amtLoans1'] / city_tracts_years_df['CFS01'])
city_tracts_years_df = city_tracts_years_df.assign(nLoans100k_per_sbjob = city_tracts_years_df['nLoans100k'] / city_tracts_years_df['CFS01'])
city_tracts_years_df = city_tracts_years_df.assign(amtLoans100k_per_sbjob = city_tracts_years_df['amtLoans100k'] / city_tracts_years_df['CFS01'])
city_tracts_years_df = city_tracts_years_df.assign(nLoans250k_per_sbjob = city_tracts_years_df['nLoans250k'] / city_tracts_years_df['CFS01'])
city_tracts_years_df = city_tracts_years_df.assign(amtLoans250k_per_sbjob = city_tracts_years_df['amtLoans250k'] / city_tracts_years_df['CFS01'])
city_tracts_years_df = city_tracts_years_df.assign(nLoansToSmallest_per_sbjob = city_tracts_years_df['nLoansToSmallest'] / city_tracts_years_df['CFS01'])
city_tracts_years_df = city_tracts_years_df.assign(amtLoansToSmallest_per_sbjob = city_tracts_years_df['amtLoansToSmallest'] / city_tracts_years_df['CFS01'])
city_tracts_years_df = city_tracts_years_df.assign(nLoans_per_sbjob = city_tracts_years_df['nLoans'] / city_tracts_years_df['CFS01'])
city_tracts_years_df = city_tracts_years_df.assign(amtLoans_per_sbjob = city_tracts_years_df['amtLoans'] / city_tracts_years_df['CFS01'])
city_tracts_years_df = city_tracts_years_df.assign(nWorkingLoans_per_sbjob = city_tracts_years_df['nWorkingLoans'] / city_tracts_years_df['CFS01'])
city_tracts_years_df = city_tracts_years_df.assign(amtWorkingLoans_per_sbjob = city_tracts_years_df['amtWorkingLoans'] / city_tracts_years_df['CFS01'])

print('recoding infinite values to NaN...')

per_job_columns = ['nLoans1_per_totaljob', 'amtLoans1_per_totaljob', 'nLoans100k_per_totaljob']
per_job_columns += ['amtLoans100k_per_totaljob', 'nLoans250k_per_totaljob', 'amtLoans250k_per_totaljob']
per_job_columns += ['nLoansToSmallest_per_totaljob', 'amtLoansToSmallest_per_totaljob']
per_job_columns += ['nLoans_per_totaljob', 'amtLoans_per_totaljob', 'nWorkingLoans_per_totaljob']
per_job_columns += ['amtWorkingLoans_per_totaljob', 'nLoans1_per_sbjob', 'amtLoans1_per_sbjob']
per_job_columns += ['nLoans100k_per_sbjob', 'amtLoans100k_per_sbjob', 'nLoans250k_per_sbjob']
per_job_columns += ['amtLoans250k_per_sbjob', 'nLoansToSmallest_per_sbjob', 'amtLoansToSmallest_per_sbjob']
per_job_columns += ['nLoans_per_sbjob', 'amtLoans_per_sbjob', 'nWorkingLoans_per_sbjob']
per_job_columns += ['amtWorkingLoans_per_sbjob']

for x in city_tracts_years_df[per_job_columns]:
    city_tracts_years_df.loc[city_tracts_years_df[x] == np.inf, x] = np.nan

print('Done!')
#city_tracts_years_df.sample(2)

    

Calulating loans per job (total and with firm size 0-19)...
recoding infinite values to NaN...
Done!


# Get ACS data

In [10]:
md_acs5_df = pandas.DataFrame()
datafiles = [census_dir+x for x in os.listdir(census_dir) if (('tracts' in x) and ('{0:}'.format(state_abbrev) in x) and ('alldata' not in x))]

for thisfile in datafiles:
    xdf = pandas.read_csv(thisfile, encoding='utf-8', low_memory=False)
    xdf = xdf.assign(year = int(thisfile[-10:-6]))
    md_acs5_df = md_acs5_df.append(xdf)

column_order = md_acs5_df.columns[0:1].tolist()
column_order += md_acs5_df.columns[-1:].tolist()
column_order += md_acs5_df.columns[1:-1].tolist()
###column_order += md_acs5_df.columns[-3:].tolist()

md_acs5_df = md_acs5_df[column_order]
print('keeping only Baltimore City tracts...')
md_acs5_df = md_acs5_df[md_acs5_df['GEOID'].apply(lambda x: 'US24510' in x)]

print('converting to numeric...')
for x in md_acs5_df.columns[4:-2]:
    md_acs5_df.loc[:, x] = pandas.to_numeric(md_acs5_df[x], errors='coerce')

print('calculating census tract numbers...')
md_acs5_df.loc[:, 'census_tract'] = md_acs5_df['GEOID'].apply(lambda x: x[12:18])
md_acs5_df.loc[:, 'census_tract'] = pandas.to_numeric(md_acs5_df['census_tract'], errors='coerce')
md_acs5_df.loc[:, 'census_tract'] = md_acs5_df['census_tract'].apply(lambda x: x/100) # get right decimalization of census tracts
md_acs5_df = md_acs5_df.assign(block_group = np.nan)
md_acs5_df.loc[md_acs5_df['GEOID'].apply(lambda x:len(x)) == 19, 
            'block_group'] = md_acs5_df['GEOID'][md_acs5_df['GEOID'].apply(lambda x:len(x)) == 19].apply(lambda x: x[18])
md_acs5_df.loc[:, 'block_group'] = pandas.to_numeric(md_acs5_df['block_group'], errors='coerce')

print('keeping 2011-2017...')
md_acs5_df = md_acs5_df[md_acs5_df['year'] >= 2011]

print('dropping block groups to avoid double-counting...')
md_acs5_df = md_acs5_df[md_acs5_df['block_group'].isnull()]  # select only census tracts (ignore block groups) to avoid double-counting
md_acs5_df = md_acs5_df.drop('block_group', axis=1)

md_acs5_df = md_acs5_df.rename(columns={'year': 'activity_year'})
md_acs5_df = md_acs5_df.set_index(['census_tract', 'activity_year'])
md_acs5_df = md_acs5_df.sort_index()
#md_acs5_df.reset_index()
#baltimore_agg_loans_df.reset_index()
city_tracts_years_df = city_tracts_years_df.reset_index().merge(md_acs5_df.reset_index(), how='left', on=['census_tract', 'activity_year']).set_index(['census_tract', 'activity_year'])
print('ok')
#city_tracts_years_df.sample(2).T

keeping only Baltimore City tracts...
converting to numeric...
calculating census tract numbers...
keeping 2011-2017...
dropping block groups to avoid double-counting...
ok


## Calculate composite demographic columns

In [11]:
print('\ncalculating and renaming estimates columns for IVs...')

if (debug >= 1):
    print('...high school graduates or higher 25 years and older...')
h = city_tracts_years_df['B15002_011'] + city_tracts_years_df['B15002_012'] + city_tracts_years_df['B15002_013'] 
h += city_tracts_years_df['B15002_014'] + city_tracts_years_df['B15002_015'] + city_tracts_years_df['B15002_016']
h += city_tracts_years_df['B15002_017'] + city_tracts_years_df['B15002_018']
h += city_tracts_years_df['B15002_028'] + city_tracts_years_df['B15002_029'] + city_tracts_years_df['B15002_030'] 
h += city_tracts_years_df['B15002_031'] + city_tracts_years_df['B15002_032'] + city_tracts_years_df['B15002_033'] 
h += city_tracts_years_df['B15002_034'] + city_tracts_years_df['B15002_035']
city_tracts_years_df = city_tracts_years_df.assign(hs_grad_25plus = pandas.to_numeric(h, errors='coerce'))

if (debug >= 1):
    print('...householder sex & race, unempoyment, poverty, home value, home age...')
city_tracts_years_df = city_tracts_years_df.rename(columns = {     
    'B11001_006': 'female_householder',
    'B11001A_001': 'white_householder',
    'B11001B_001': 'black_householder',
    'B23025_005': 'unemployed_16plus',
    'B17001_002': 'poverty_past_12_months',
    'B25077_001': 'median_home_value',
    'B25035_001': 'median_year_built'
})

if (debug >= 1):
    print('...total householders...')
city_tracts_years_df = city_tracts_years_df.assign(total_householders = pandas.to_numeric(
                                             (city_tracts_years_df['B11001_002'] + city_tracts_years_df['B11001_007']
                                             ), errors='coerce'
                                         )
                                        )

if (debug >= 1):
    print('...race, owner-occupied units, mfi...')
city_tracts_years_df = city_tracts_years_df.rename(columns = {
    'B02001_002': 'pop_white',
    'B02001_003': 'pop_black',
    'B25003_002': 'owner_occ_housing_units',
    'B19113_001': 'mfi'    
})
if (debug >= 1):
    print('....comparison variables: total population, total households, poverty status...')
city_tracts_years_df = city_tracts_years_df.rename(columns = {
    'B01001_001': 'pop_total',
    'B23025_002': 'labor_force_16plus',
    'B17001_001': 'poverty_status_known'
})

if (debug >= 1):
    print('...population 25plus...')
city_tracts_years_df = city_tracts_years_df.assign(pop_25plus = pandas.to_numeric(
                                             (city_tracts_years_df['B01001_011'] + city_tracts_years_df['B01001_012'] + city_tracts_years_df['B01001_013'] 
                                              + city_tracts_years_df['B01001_014'] + city_tracts_years_df['B01001_015'] + city_tracts_years_df['B01001_016']
                                              + city_tracts_years_df['B01001_017'] + city_tracts_years_df['B01001_018'] + city_tracts_years_df['B01001_019']
                                              + city_tracts_years_df['B01001_020'] + city_tracts_years_df['B01001_021'] + city_tracts_years_df['B01001_022']
                                              + city_tracts_years_df['B01001_023'] + city_tracts_years_df['B01001_024'] + city_tracts_years_df['B01001_025']
                                              + city_tracts_years_df['B01001_035'] + city_tracts_years_df['B01001_036'] + city_tracts_years_df['B01001_037']
                                              + city_tracts_years_df['B01001_038'] + city_tracts_years_df['B01001_039'] + city_tracts_years_df['B01001_040']
                                              + city_tracts_years_df['B01001_041'] + city_tracts_years_df['B01001_042'] + city_tracts_years_df['B01001_043']
                                              + city_tracts_years_df['B01001_044'] + city_tracts_years_df['B01001_045'] + city_tracts_years_df['B01001_046']
                                              + city_tracts_years_df['B01001_047'] + city_tracts_years_df['B01001_048'] + city_tracts_years_df['B01001_049']
                                             ), errors='coerce'
                                         )
                                        )
#city_tracts_years_df.sample(1).T

print('Done!')
#city_tracts_years_df


calculating and renaming estimates columns for IVs...
...high school graduates or higher 25 years and older...
...householder sex & race, unempoyment, poverty, home value, home age...
...total householders...
...race, owner-occupied units, mfi...
....comparison variables: total population, total households, poverty status...
...population 25plus...
Done!


## Get errors for composite columns

### Create functions

In [12]:
### Guide on how to calculate errors in percentages:
# https://www.census.gov/content/dam/Census/library/publications/2018/acs/acs_general_handbook_2018_ch08.pdf
    
## Aggregating Data Across Population Subgroups: add error for each group in quadrature, divide by 1.645 for serr

def find_serr_hsgrad25plus(row):
    return pandas.to_numeric(np.sqrt(row['B15002_011_err']**2 + row['B15002_012_err']**2 + row['B15002_013_err']**2 
                                 + row['B15002_014_err']**2 + row['B15002_015_err']**2 + row['B15002_016_err']**2 
                                 + row['B15002_017_err']**2 + row['B15002_018_err']**2 + 
                                 + row['B15002_028_err']**2 + row['B15002_029_err']**2 + row['B15002_030_err']**2 
                                 + row['B15002_031_err']**2 + row['B15002_032_err']**2 + row['B15002_033_err']**2 
                                 + row['B15002_034_err']**2 + row['B15002_035_err']**2
                                ) / 1.645, errors='coerce')

def find_serr_householders(row):
    return pandas.to_numeric(np.sqrt(row['B11001_002_err']**2 + row['B11001_007_err']**2 
                                ) / 1.645, errors='coerce')

def find_serr_pop25plus(row):
    return pandas.to_numeric(np.sqrt(row['B01001_011_err']**2 + row['B01001_012_err']**2 + row['B01001_013_err']**2 
                                     + row['B01001_014_err']**2 + row['B01001_015_err']**2 + row['B01001_016_err']**2 
                                     + row['B01001_017_err']**2 + row['B01001_018_err']**2 + row['B01001_019_err']**2 
                                     + row['B01001_020_err']**2 + row['B01001_021_err']**2 + row['B01001_022_err']**2 
                                     + row['B01001_023_err']**2 + row['B01001_024_err']**2 + row['B01001_025_err']**2 
                                     + row['B01001_035_err']**2 + row['B01001_036_err']**2 + row['B01001_037_err']**2 
                                     + row['B01001_038_err']**2 + row['B01001_039_err']**2 + row['B01001_040_err']**2 
                                     + row['B01001_041_err']**2 + row['B01001_042_err']**2 + row['B01001_043_err']**2 
                                     + row['B01001_044_err']**2 + row['B01001_045_err']**2 + row['B01001_046_err']**2 
                                     + row['B01001_047_err']**2 + row['B01001_048_err']**2 + row['B01001_049_err']**2 
                                    ) / 1.645, errors='coerce')
print('Defined standard-error-calculating functions!')
print('ok')

Defined standard-error-calculating functions!
ok


### Calculate errors

In [13]:
print('\ncalculating and renaming margins of error columns for IVs...')

if (debug >= 1):
    print('...margins for race, owner-occupied units, mfi...')
city_tracts_years_df = city_tracts_years_df.rename(columns = {
    'B02001_002_err': 'pop_white_err',
    'B02001_003_err': 'pop_black_err',
    'B25003_002_err': 'owner_occ_housing_units_err',
    'B19113_001_err': 'mfi_err'    
})



if (debug >= 1):
    print('...standard errors for hs graduates 25 and older (using custom serr-finding function...')
city_tracts_years_df = city_tracts_years_df.assign(hs_grad_25plus_serr = pandas.to_numeric(city_tracts_years_df.apply(lambda row: find_serr_hsgrad25plus(row), axis=1), errors='coerce'))

if (debug >= 1):
    print('...margins of error for householder sex & race, unempoyment, poverty, home value, home age...')
city_tracts_years_df = city_tracts_years_df.rename(columns = {     
    'B11001_006_err': 'female_householder_err',
    'B11001A_001_err': 'black_householder_err',
    'B11001B_001_err': 'white_householder_err',
    'B23025_005_err': 'unemployed_16plus_err',
    'B17001_002_err': 'poverty_past_12_months_err',
    'B25077_001_err': 'median_home_value_err',
    'B25035_001_err': 'median_year_built_err'
})

print('\ncalculating and renaming margins of error for comparison variables...')
if (debug >= 1):
    print('...race, owner-occupied units, mfi...')
city_tracts_years_df = city_tracts_years_df.rename(columns = {
    'B01001_001_err': 'pop_total_err',
    'B17001_001_err': 'poverty_status_known_err'
})
if (debug >= 1):
    print('...population 25plus...')
city_tracts_years_df = city_tracts_years_df.assign(pop_25plus_serr = pandas.to_numeric(city_tracts_years_df.apply(lambda row: find_serr_pop25plus(row), axis=1), errors='coerce'))

if (debug >= 1):
    print('...total householders...')
city_tracts_years_df = city_tracts_years_df.assign(total_householders_serr = pandas.to_numeric(city_tracts_years_df.apply(lambda row: find_serr_householders(row), axis=1), errors='coerce'))

if (debug >= 1):
    print('...labor force, poverty status known...')
city_tracts_years_df = city_tracts_years_df.rename(columns = {
    'B23025_002_err': 'labor_force_16plus_err',
    'B17001_001_err': 'poverty_status_known_err'
})


print('dropping columns we do not care about...')
columns_do_not_care = ['B15002_011','B15002_012','B15002_013','B15002_014','B15002_015']
columns_do_not_care += ['B15002_016','B15002_017','B15002_018','B15002_028','B15002_029']
columns_do_not_care += ['B15002_030','B15002_031','B15002_032','B15002_033','B15002_034']
columns_do_not_care += ['B15002_035','B01001_011','B01001_012','B01001_013','B01001_014']
columns_do_not_care += ['B01001_015','B01001_016','B01001_017','B01001_018','B01001_019']
columns_do_not_care += ['B01001_020','B01001_021','B01001_022','B01001_023','B01001_024']
columns_do_not_care += ['B01001_025','B01001_035','B01001_036','B01001_037','B01001_038']
columns_do_not_care += ['B01001_039','B01001_040','B01001_041','B01001_042','B01001_043']
columns_do_not_care += ['B01001_044','B01001_045','B01001_046','B01001_047','B01001_048']
columns_do_not_care += ['B01001_049', 'B11001_002', 'B11001_007']
columns_do_not_care += ['B15002_011_err','B15002_012_err','B15002_013_err']
columns_do_not_care += ['B15002_014_err','B15002_015_err','B15002_016_err']
columns_do_not_care += ['B15002_017_err','B15002_018_err','B15002_028_err']
columns_do_not_care += ['B15002_029_err','B15002_030_err','B15002_031_err']
columns_do_not_care += ['B15002_032_err','B15002_033_err','B15002_034_err']
columns_do_not_care += ['B15002_035_err','B15002_011_err','B15002_012_err']
columns_do_not_care += ['B15002_013_err','B15002_014_err','B15002_015_err']
columns_do_not_care += ['B15002_016_err','B15002_017_err','B15002_018_err']
columns_do_not_care += ['B15002_028_err','B15002_029_err','B15002_030_err']
columns_do_not_care += ['B15002_031_err','B15002_032_err','B15002_033_err']
columns_do_not_care += ['B15002_034_err','B15002_035_err','B01001_011_err']
columns_do_not_care += ['B01001_012_err','B01001_013_err','B01001_014_err']
columns_do_not_care += ['B01001_015_err','B01001_016_err','B01001_017_err']
columns_do_not_care += ['B01001_018_err','B01001_019_err','B01001_020_err']
columns_do_not_care += ['B01001_021_err','B01001_022_err','B01001_023_err']
columns_do_not_care += ['B01001_024_err','B01001_025_err','B01001_035_err']
columns_do_not_care += ['B01001_036_err','B01001_037_err','B01001_038_err']
columns_do_not_care += ['B01001_039_err','B01001_040_err','B01001_041_err']
columns_do_not_care += ['B01001_042_err','B01001_043_err','B01001_044_err']
columns_do_not_care += ['B01001_045_err','B01001_046_err','B01001_047_err']
columns_do_not_care += ['B11001_002_err','B11001_007_err']

columns_do_not_care += ['B01001_048_err','B01001_049_err','STATE']
city_tracts_years_df = city_tracts_years_df.drop(columns_do_not_care, axis=1)


print('Calculated errors for all columns!')

#city_tracts_years_df.sample(2)
print('ok')


calculating and renaming margins of error columns for IVs...
...margins for race, owner-occupied units, mfi...
...standard errors for hs graduates 25 and older (using custom serr-finding function...
...margins of error for householder sex & race, unempoyment, poverty, home value, home age...

calculating and renaming margins of error for comparison variables...
...race, owner-occupied units, mfi...
...population 25plus...
...total householders...
...labor force, poverty status known...
dropping columns we do not care about...
Calculated errors for all columns!
ok


In [14]:
vars_for_percentification = ['pop_white', 'pop_black', 'black_householder', 'white_householder']
vars_for_percentification += ['owner_occ_housing_units', 'hs_grad_25plus', 'female_householder']
vars_for_percentification += ['unemployed_16plus', 'poverty_past_12_months']

vars_for_percentification += ['pop_white_err', 'pop_black_err', 'black_householder_err', 'white_householder_err']
vars_for_percentification += ['owner_occ_housing_units_err', 'hs_grad_25plus_serr', 'female_householder_err']
vars_for_percentification += ['unemployed_16plus_err', 'poverty_past_12_months_err']

vars_for_percentification += ['pop_total', 'total_householders', 'pop_25plus', 'labor_force_16plus']
vars_for_percentification += ['poverty_status_known']

vars_for_percentification += ['pop_total_err', 'total_householders_serr', 'pop_25plus_serr', 'labor_force_16plus_err']
vars_for_percentification += ['poverty_status_known_err']
vars_for_percentification
#city_tracts_years_df[vars_for_percentification].columns.tolist()
print('ok')

ok


# Calculate percentages for needed demographic variables

In [15]:

#[x for x in vars_for_percentification if "_err" not in x]

city_tracts_years_df = city_tracts_years_df.assign(pct_white = pandas.to_numeric((city_tracts_years_df['pop_white'] / city_tracts_years_df['pop_total']), errors='coerce'))
city_tracts_years_df = city_tracts_years_df.assign(pct_black = pandas.to_numeric((city_tracts_years_df['pop_black'] / city_tracts_years_df['pop_total']), errors='coerce'))

city_tracts_years_df = city_tracts_years_df.assign(pct_white_householders = pandas.to_numeric((city_tracts_years_df['white_householder'] / city_tracts_years_df['total_householders']), errors='coerce'))
city_tracts_years_df = city_tracts_years_df.assign(pct_black_householders = pandas.to_numeric((city_tracts_years_df['black_householder'] / city_tracts_years_df['total_householders']), errors='coerce'))
city_tracts_years_df = city_tracts_years_df.assign(pct_female_householders = pandas.to_numeric((city_tracts_years_df['female_householder'] / city_tracts_years_df['total_householders']), errors='coerce'))

city_tracts_years_df = city_tracts_years_df.assign(pct_hs_grad = pandas.to_numeric(city_tracts_years_df['hs_grad_25plus'], errors='coerce') / pandas.to_numeric(city_tracts_years_df['pop_25plus'], errors='coerce'))
city_tracts_years_df = city_tracts_years_df.assign(pct_unemployed = pandas.to_numeric(city_tracts_years_df['unemployed_16plus'], errors='coerce') / pandas.to_numeric(city_tracts_years_df['labor_force_16plus'], errors='coerce'))
city_tracts_years_df = city_tracts_years_df.assign(pct_unemployed = pandas.to_numeric(city_tracts_years_df['poverty_past_12_months'], errors='coerce') / pandas.to_numeric(city_tracts_years_df['poverty_status_known'], errors='coerce'))

print('ok')
#city_tracts_years_df.columns.tolist()

ok


## Function to calculate errors in percentages

In [16]:
#Guide on how to do this:
#### https://www.census.gov/content/dam/Census/library/publications/2018/acs/acs_general_handbook_2018_ch08.pdf

# X and Y are the measured values (not the errors) - X for the subsgroup and Y for the whole sample
# Let P = X/Y  (the proportion we calculated in the last step)
# dX and dY are the measured errors
# dP = (1/Y) * np.sqrt(dX**2 - (P**2 * dY**2))
# Standard error of P is dP/1.645
#### this calculation is done verbosely in fnid_pop_white_serr, quickly in other functions

def find_errors_in_pct(X, Y, dX, dY, verboselevel = 0):
    try:
        P = X / Y
        oneoverY = 1 / Y
        dXsq = dX**2
        dYsq = dY**2
        Psq = P**2
        PsqdYsq = Psq * dYsq
        if (PsqdYsq <= dXsq):
            underroot = dXsq - PsqdYsq
        else:
            underroot = dXsq + PsqdYsq
        rooty = np.sqrt(underroot)
        dP = oneoverY * rooty
        SE = dP / 1.645
        if (verboselevel >= 2):
#            print('X = pop_white, Y = pop_total')
            print('X = {0:.0f}, dX = {1:.0f} ({2:.1%} error)'.format(X, dX, dX/X))
            print('Y = {0:.0f}, dY = {1:.0f} ({2:.1%} error)'.format(Y, dY, dY/Y))
        if (verboselevel >= 3):
            print('P = {0:.3f}'.format(P))
            print('dXsq = {0:.0f}, dYsq = {1:.0f}, Psq = {2:.3f}'.format(dXsq, dYsq, Psq))
            print('PsqdYsq = {0:.0f}, underroot = {1:.0f}, rooty = {2:.3f}'.format(PsqdYsq, underroot, rooty))
            print('dP = {0:.3f}'.format(dP))
            print('SE = {0:.3f}'.format(SE))
        if (verboselevel >= 2):
            print('RESULT: {0:.2%} +/- {1:.2%}'.format(P, SE)) 
            print('\n')
        return pandas.to_numeric(SE, errors='coerce')
    except ZeroDivisionError:
        return np.nan
print('Defined functions to calculate standard errors in percentages!')
print('ok')

Defined functions to calculate standard errors in percentages!
ok


### Calculate errors in percentage values

In [17]:
verboselevel = 0

print('Calculating errors in percentages...')
city_tracts_years_df = city_tracts_years_df.assign(pct_white_serr = np.nan)
city_tracts_years_df = city_tracts_years_df.assign(pct_black_serr = np.nan)
city_tracts_years_df = city_tracts_years_df.assign(pct_white_householders_serr = np.nan)
city_tracts_years_df = city_tracts_years_df.assign(pct_black_householders_serr = np.nan)
city_tracts_years_df = city_tracts_years_df.assign(pct_female_householders_serr = np.nan)
city_tracts_years_df = city_tracts_years_df.assign(pct_hs_grad_serr = np.nan)
city_tracts_years_df = city_tracts_years_df.assign(pct_unemployed_serr = np.nan)
city_tracts_years_df = city_tracts_years_df.assign(pct_poverty_serr = np.nan)

#city_tracts_years_df.loc[:, 
#              'poverty_status_known_last12months_total_err'] = pandas.to_numeric(city_tracts_years_df['poverty_status_known_last12months_total_err'], errors='coerce')


for ix, thisrow in city_tracts_years_df.iterrows():
    if (verboselevel >= 2):
        print('Census tract {0:}...'.format(ix))
    #print('pct_white_serr...')
    city_tracts_years_df.loc[ix, 'pct_white_serr'] = find_errors_in_pct(thisrow['pop_white'], thisrow['pop_total'], thisrow['pop_white_err'], thisrow['pop_total_err'], verboselevel)
    #print('pct_black_serr...')
    city_tracts_years_df.loc[ix, 'pct_black_serr'] = find_errors_in_pct(thisrow['pop_black'], thisrow['pop_total'], thisrow['pop_black_err'], thisrow['pop_total_err'], verboselevel)
    #print('pct_white_householders_serr...')
    city_tracts_years_df.loc[ix, 'pct_white_householders_serr'] = find_errors_in_pct(thisrow['white_householder'], thisrow['total_householders'], thisrow['white_householder_err'], thisrow['total_householders_serr'], verboselevel)
    city_tracts_years_df.loc[ix, 'pct_black_householders_serr'] = find_errors_in_pct(thisrow['white_householder'], thisrow['total_householders'], thisrow['white_householder_err'], thisrow['total_householders_serr'], verboselevel)
    city_tracts_years_df.loc[ix, 'pct_female_householders_serr'] = find_errors_in_pct(thisrow['white_householder'], thisrow['total_householders'], thisrow['white_householder_err'], thisrow['total_householders_serr'], verboselevel)
    
    city_tracts_years_df.loc[ix, 'pct_hs_grad_serr'] = find_errors_in_pct(thisrow['hs_grad_25plus'], thisrow['pop_25plus'], thisrow['hs_grad_25plus_serr'], thisrow['pop_25plus_serr'], verboselevel)
    city_tracts_years_df.loc[ix, 'pct_unemployed_serr'] = find_errors_in_pct(thisrow['unemployed_16plus'], thisrow['labor_force_16plus'], thisrow['hs_grad_25plus_serr'], thisrow['labor_force_16plus_err'], verboselevel)
    city_tracts_years_df.loc[ix, 'pct_poverty_serr'] = find_errors_in_pct(thisrow['poverty_past_12_months'], thisrow['poverty_status_known'], thisrow['poverty_past_12_months_err'], thisrow['poverty_status_known_err'], verboselevel)

if (verboselevel >= 1):
    for ix, thisrow in city_tracts_years_df.iterrows():
        print('Census tract {0:,.0f}'.format(ix))
        print('{0:,.0f} +/- {1:,.0f} white'.format(
            thisrow['pop_white'], thisrow['pop_white_err']
        ))
        print('{0:,.0f} +/- {1:,.0f} total'.format(
            thisrow['pop_total'], thisrow['pop_total_err']
        ))
        print('{0:.1%} +/- {1:.1%}'.format(
            thisrow['pct_white'], thisrow['pct_white_serr']
        ))
        print('\n')
print('ok')

Calculating errors in percentages...
ok


# Correct for inflation

In [18]:
money_columns = ['amtLoans1', 'amtLoans100k', 'amtLoans250k', 'amtLoansToSmallest']
money_columns += ['amtLoans', 'amtWorkingLoans']
money_columns += ['mfi', 'median_home_value']
money_columns += ['amtLoans1_per_totaljob', 'amtLoans100k_per_totaljob']
money_columns += ['amtLoans250k_per_totaljob', 'amtLoansToSmallest_per_totaljob']
money_columns += ['amtLoans_per_totaljob', 'amtWorkingLoans_per_totaljob']
money_columns += ['amtLoans1_per_sbjob', 'amtLoans100k_per_sbjob']
money_columns += ['amtLoans250k_per_sbjob', 'amtLoansToSmallest_per_sbjob']
money_columns += ['amtLoans_per_sbjob', 'amtWorkingLoans_per_sbjob']

print('getting inflation data...')
cpi_1913_2017_df = pandas.read_csv(inflation_dir+'cpi-1913-2017.csv', index_col='Year')
cpi_annual_s = cpi_1913_2017_df['Jan']
cpi_annual_s.name = 'rawfactor'
value_in_2017 = cpi_annual_s.loc[2017]

annual_inflator_s = 1 / (cpi_annual_s / value_in_2017)
print('inflating pre-2017 monetary values...')
inflate_these_df = city_tracts_years_df[money_columns]
newcolnames = [x+'_adj' for x in inflate_these_df.columns.tolist()]
inflate_these_df.columns = newcolnames

inflated_df = pandas.DataFrame()
for i in inflate_these_df.index.get_level_values(1).drop_duplicates().tolist():
    inflated_df_i = inflate_these_df.xs(i, level=1).apply(lambda x: x * annual_inflator_s.loc[i])
    inflated_df_i['activity_year'] = i
    inflated_df = pandas.concat((inflated_df, inflated_df_i), axis=0)
inflated_df = inflated_df.reset_index().set_index(['census_tract', 'activity_year'])

print('joining inflated money columns to the rest of the columns...')
city_tracts_years_df = city_tracts_years_df.join(inflated_df, how='left')

print('Done!')
#baltimore_tracts_years_df[['amtWorkingLoans_per_sbjob', 'amtWorkingLoans_per_sbjob_adj', 'mfi', 'mfi_adj']].sample(3).T

getting inflation data...
inflating pre-2017 monetary values...
joining inflated money columns to the rest of the columns...
Done!


# Add codes (e.g. CSA, income group...)

In [19]:
# Add community statistical area names
print('Matching community statistical areas...')
if (city == 'Baltimore'):
    tract_to_csa_df = pandas.read_csv(code_lookup_dir+'census_tract_to_neighborhood.csv', index_col='NAME10')
    city_tracts_years_df = city_tracts_years_df.reset_index().merge(tract_to_csa_df.reset_index(), how='left', left_on='census_tract', right_on='NAME10').set_index(['census_tract', 'activity_year'])
    city_tracts_years_df = city_tracts_years_df.drop(['NAME10', 'TRACTCE10', 'GEOID10'], axis=1)

print('Decoding income group names...')
# Get income group names
city_tracts_years_df = city_tracts_years_df.rename(columns = {'income_group_total': 'income_group_code'})
city_tracts_years_df = city_tracts_years_df.assign(income_group = np.nan)

city_tracts_years_df.loc[city_tracts_years_df['income_group_code'] == 1, 'income_group'] = '< 10% of Median Family Income (MFI)'
city_tracts_years_df.loc[city_tracts_years_df['income_group_code'] == 2, 'income_group'] = '10% to 20% of MFI'
city_tracts_years_df.loc[city_tracts_years_df['income_group_code'] == 3, 'income_group'] = '20% to 30% of MFI'
city_tracts_years_df.loc[city_tracts_years_df['income_group_code'] == 4, 'income_group'] = '30% to 40% of MFI'
city_tracts_years_df.loc[city_tracts_years_df['income_group_code'] == 5, 'income_group'] = '40% to 50% of MFI'
city_tracts_years_df.loc[city_tracts_years_df['income_group_code'] == 6, 'income_group'] = '50% to 60% of MFI'
city_tracts_years_df.loc[city_tracts_years_df['income_group_code'] == 7, 'income_group'] = '60% to 70% of MFI'
city_tracts_years_df.loc[city_tracts_years_df['income_group_code'] == 8, 'income_group'] = '70% to 80% of MFI'
city_tracts_years_df.loc[city_tracts_years_df['income_group_code'] == 9, 'income_group'] = '80% to 90% of MFI'
city_tracts_years_df.loc[city_tracts_years_df['income_group_code'] == 10, 'income_group'] = '90% to 100% of MFI'
city_tracts_years_df.loc[city_tracts_years_df['income_group_code'] == 11, 'income_group'] = '100% to 110% of MFI'
city_tracts_years_df.loc[city_tracts_years_df['income_group_code'] == 12, 'income_group'] = '110% to 120% of MFI'
city_tracts_years_df.loc[city_tracts_years_df['income_group_code'] == 13, 'income_group'] = '> 120% of MFI'

print('Decoding income levels (low/moderate/middle/upper/unknown)...')
# Get levels (low, moderate, middle, upper)
city_tracts_years_df = city_tracts_years_df.assign(cra_level = np.nan)
city_tracts_years_df.loc[(city_tracts_years_df['income_group_code'] >= 1) & (city_tracts_years_df['income_group_code'] <= 5), 'cra_level'] = 'low'
city_tracts_years_df.loc[(city_tracts_years_df['income_group_code'] >= 6) & (city_tracts_years_df['income_group_code'] <= 8), 'cra_level'] = 'moderate'
city_tracts_years_df.loc[(city_tracts_years_df['income_group_code'] >= 9) & (city_tracts_years_df['income_group_code'] <= 12), 'cra_level'] = 'middle'
city_tracts_years_df.loc[(city_tracts_years_df['income_group_code'] == 13), 'cra_level'] = 'upper'
city_tracts_years_df.loc[(city_tracts_years_df['income_group_code'] == 14), 'cra_level'] = 'unknown'

print('Writing out...')
city_tracts_years_df.to_csv(city_data_dir+'{0:}_alldata.csv'.format(cityname_file), encoding='utf-8')
print('Done!')

Matching community statistical areas...
Decoding income group names...
Decoding income levels (low/moderate/middle/upper/unknown)...
Writing out...
Done!
